In [1]:
from pathlib import Path, PurePath
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
from analysis import read_behaviors, read_moea_fitness, unpickle_moea
from utility.util_functions import calc_str_disparity, euclidean_distance
from utility.visualize import plot_path
import pickle
import numpy as np
from deap import base, creator
init_notebook_mode(connected=True)


In [26]:
# get the path for all behavioral features
files = sorted(Path('data/results/').glob('transferability_*/behavioral_features.dat'))
fitness_f = sorted(Path('data/results/').glob('transferability_*/fitness.dat'))

In [27]:
columns = [
        'avg_left', 'avg_right',
        's1', 's2', 's3', 's4', 's5', 's6', 's7',
        'area0_percentage',
        'area1_percentage',
        'area2_percentage',
    ]

In [28]:
fitness_f

[PosixPath('data/results/transferability_simulation_1/fitness.dat'),
 PosixPath('data/results/transferability_simulation_2/fitness.dat'),
 PosixPath('data/results/transferability_simulation_3/fitness.dat'),
 PosixPath('data/results/transferability_simulation_4/fitness.dat'),
 PosixPath('data/results/transferability_simulation_5/fitness.dat'),
 PosixPath('data/results/transferability_simulation_6/fitness.dat'),
 PosixPath('data/results/transferability_simulation_7/fitness.dat')]

In [29]:
behaviors = read_behaviors([files[6]])[0]

In [30]:
behaviors.head()

,gen,genome_id,simulation,avg_left,avg_right,s1,s2,s3,s4,s5,s6,s7,area0_percentage,area1_percentage,area2_percentage
0,0,1,VREP,0.504093,0.539303,0.000000,0.000000,0.00000,0.000000,0.000000,0.995000,0.995000,1.0,0.0,0.0
1,0,1,THYMIO,0.518615,0.549875,0.310606,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0
2,0,1,VREP,0.504093,0.539303,0.000000,0.000000,0.00000,0.000000,0.000000,0.995000,0.995000,1.0,0.0,0.0
3,0,2,VREP,0.212503,0.378413,0.000000,0.000000,0.00000,0.000000,0.000000,0.990826,0.990826,1.0,0.0,0.0
4,0,3,VREP,0.736711,0.637441,0.410985,0.462121,0.42803,0.359848,0.287879,0.085227,0.115530,1.0,0.0,0.0


In [31]:
b_grouped = behaviors.groupby(['genome_id']).filter(lambda x: len(x) > 1).groupby(['genome_id', 'simulation']).first()

In [47]:
b_grouped

gen  avg_left  avg_right        s1        s2        s3  \
genome_id simulation                                                           
1         THYMIO        0  0.518615   0.549875  0.310606  0.000000  0.000000   
          VREP          0  0.504093   0.539303  0.000000  0.000000  0.000000   
21        THYMIO        1  0.545113   0.543098  0.000000  0.000000  0.000000   
          VREP          1  0.560477   0.559591  0.000000  0.000000  0.000000   
22        THYMIO        1  0.378983   0.031678  0.000000  0.000000  0.131313   
          VREP          1  0.354143   0.038121  0.000000  0.000000  0.000000   
23        THYMIO        1  0.267975   0.742957  0.366057  0.470882  0.399334   
          VREP          1  0.277488   0.699885  0.000000  0.000000  0.000000   
24        THYMIO        1  0.657974   0.490291  0.088889  0.211111  0.511111   
          VREP          1  0.776610   0.481487  0.000000  0.000000  0.000000   
25        THYMIO        1  0.672193   0.366750  0.080515  0.299517  0.090177   
          VREP          1  0.691909   0.388507  0.000000  0.000000  0.000000   
26        THYMIO        1  0.248848   0.114475  0.000000  0.000000  0.000000   
          VREP          1  0.313331   0.127141  0.000000  0.000000  0.000000   
27        THYMIO        1  0.011271   0.011493  0.000000  0.000000  0.000000   
          VREP          1  0.017542   0.014921  0.000000  0.000000  0.000000   
28        THYMIO        1  0.012912   0.019187  0.000000  0.000000  0.000000   
          VREP          1  0.008554   0.040431  0.000000  0.000000  0.000000   
29        THYMIO        1  0.226414   0.958640  0.576271  0.830508  0.728814   
          VREP          1  0.146439   0.969218  0.270000  0.330000  0.330000   
30        THYMIO        1  0.300338   0.100790  0.000000  0.000000  0.000000   
          VREP          1  0.282995   0.091388  0.000000  0.000000  0.000000   
31        THYMIO        1  0.303167   0.853599  0.249201  0.496805  0.400958   
          VREP          1  0.305073   0.849448  0.065000  0.000000  0.000000   
32        THYMIO        1  0.041928   0.365912  0.000000  0.000000  0.000000   
          VREP          1  0.053659   0.357635  0.000000  0.000000  0.000000   
33        THYMIO        1  0.807875   0.254004  0.344156  0.543831  0.384740   
          VREP          1  0.779519   0.280390  0.175000  0.030000  0.000000   
34        THYMIO        1  0.845278   0.166906  0.419250  0.517129  0.502447   
          VREP          1  0.888804   0.121033  0.310000  0.305000  0.315000   
...                   ...       ...        ...       ...       ...       ...   
275       THYMIO       13  0.019335   0.401517  0.372340  0.372340  0.271277   
          VREP         13  0.023211   0.364502  0.000000  0.000000  0.000000   
277       THYMIO       13  0.571051   0.527891  0.956604  0.945283  0.928302   
          VREP         13  0.477635   0.658948  0.000000  0.000000  0.000000   
284       THYMIO       14  0.739171   0.874739  0.000000  0.000000  0.093023   
          VREP         14  0.799554   0.927195  0.232859  0.166882  0.186287   
298       THYMIO       14  0.868679   0.967028  0.000000  0.148649  0.229730   
          VREP         14  0.846100   0.967835  0.228369  0.090780  0.104965   
310       THYMIO       15  0.565095   0.062640  0.233333  0.346667  0.346667   
          VREP         15  0.475782   0.085337  0.310000  0.260000  0.000000   
315       THYMIO       15  0.574857   0.000225  0.000000  0.045455  0.090909   
          VREP         15  0.465183   0.000103  0.490826  0.385321  0.229358   
316       THYMIO       15  0.811769   0.867967  0.000000  0.161290  0.225806   
          VREP         15  0.791506   0.855264  0.125874  0.115385  0.068765   
318       THYMIO       15  0.428237   0.058729  0.000000  0.000000  0.000000   
          VREP         15  0.468393   0.053142  0.387435  0.240838  0.000000   
321       THYMIO       16  0.955306   0.880228  0.702048  0.724395  0.737430   
          VREP        

In [43]:
euclidean_distance(b_grouped[columns].iloc[1], b_grouped[columns].iloc[0])

1.4411276005755036

In [68]:
b_grouped[columns].iloc[1].name[0]

1

In [72]:
for i in range(1, len(b_grouped), 2):
    print('genome_id:', 
          b_grouped[columns].iloc[i].name[0],
          'actual diversity',
          euclidean_distance(
              b_grouped[columns].iloc[i],
              b_grouped[columns].iloc[i-1]
          )
    )

genome_id: 1 actual diversity 1.4411276005755036
genome_id: 21 actual diversity 1.28233179689525
genome_id: 22 actual diversity 0.8354418137129416
genome_id: 23 actual diversity 0.8949282396696494
genome_id: 24 actual diversity 1.357375627360323
genome_id: 25 actual diversity 0.5902130124282489
genome_id: 26 actual diversity 0.6542986261218409
genome_id: 27 actual diversity 0.027146576416422197
genome_id: 28 actual diversity 0.034384883575712406
genome_id: 29 actual diversity 0.8599463873152332
genome_id: 30 actual diversity 0.027838293698098676
genome_id: 31 actual diversity 0.7723275592853389
genome_id: 32 actual diversity 0.6785288730953947
genome_id: 33 actual diversity 0.8449155497327926
genome_id: 34 actual diversity 0.44806306459681555
genome_id: 35 actual diversity 0.3424180102581236
genome_id: 36 actual diversity 1.573231935259447
genome_id: 37 actual diversity 0.5335242325238344
genome_id: 38 actual diversity 0.4749042927730572
genome_id: 39 actual diversity 0.122979068812027

In [ ]:
fitness_data = read_moea_fitness([fitness_f[6]])[0]

In [ ]:
fitness_data.head()

In [ ]:
hof = unpickle_moea('data/results/transferability_simulation_7/hof')

In [ ]:
len(hof)

In [ ]:
trace1 = go.Scatter3d(
    x=fitness_data.loc[:, 'fitness'],
    y=fitness_data.loc[:, 'str_disparity'],
    z=fitness_data.loc[:, 'diversity'],
    mode='markers',
    marker=dict(
        size=8,
        color=fitness_data.loc[:, 'diversity'], # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    ),
    text=fitness_data.loc[:, 'genome_id'],
)

data = [trace1]
layout = go.Layout(
    title='Evaluation objectives. MOEA. Transferability.',
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    ),
    scene = dict(
        xaxis = dict(
            title='Task-fitness'),
        yaxis = dict(
            title='STR Disparity'),
        zaxis = dict(
            title='Diversity'),
        annotations= [dict(
            showarrow = True,
            x = ind.fitness.values[0],
            y = ind.fitness.values[1],
            z = ind.fitness.values[2],
            text = ind.key,
            xanchor = "left",
            xshift = 10,
            opacity = 0.7,
            textangle = 0,
            ax = 0,
            ay = -75,
            font = dict(
              color = "black",
              size = 12
            ),
            arrowcolor = "black",
            arrowsize = 3,
            arrowwidth = 1,
            arrowhead = 1
          ) for ind in hof
        ]
    ),
    showlegend=True
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='3d-scatter-colorscale')

In [ ]:
for ind in hof:
    print(ind.key, ind.fitness.values)

In [2]:
genomes_file = sorted(Path('data/results/transferability_simulation_7/deap_inds').glob('*_genome.pkl'))
transfered_file = sorted(Path('data/results/transferability_simulation_7/deap_inds').glob('*_transformed_genome_.pkl'))
genomes = [unpickle_moea(genome) for genome in genomes_file]
transferred = [unpickle_moea(genome) for genome in transfered_file]

/Users/richban/.local/share/virtualenvs/behavioral.neuroevolution-ViNSkuNA/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning:

A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.

/Users/richban/.local/share/virtualenvs/behavioral.neuroevolution-ViNSkuNA/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning:

A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.



In [8]:
transfered_file

[PosixPath('data/results/transferability_simulation_7/deap_inds/100_transformed_genome_.pkl'),
 PosixPath('data/results/transferability_simulation_7/deap_inds/101_transformed_genome_.pkl'),
 PosixPath('data/results/transferability_simulation_7/deap_inds/102_transformed_genome_.pkl'),
 PosixPath('data/results/transferability_simulation_7/deap_inds/119_transformed_genome_.pkl'),
 PosixPath('data/results/transferability_simulation_7/deap_inds/130_transformed_genome_.pkl'),
 PosixPath('data/results/transferability_simulation_7/deap_inds/139_transformed_genome_.pkl'),
 PosixPath('data/results/transferability_simulation_7/deap_inds/141_transformed_genome_.pkl'),
 PosixPath('data/results/transferability_simulation_7/deap_inds/145_transformed_genome_.pkl'),
 PosixPath('data/results/transferability_simulation_7/deap_inds/147_transformed_genome_.pkl'),
 PosixPath('data/results/transferability_simulation_7/deap_inds/157_transformed_genome_.pkl'),
 PosixPath('data/results/transferability_simulatio

In [3]:
g = sorted(genomes, key=lambda x: x.key)

In [4]:
t = sorted(transferred, key=lambda x: x.key)

In [22]:
sim_genome = list(filter(lambda x: x.key == 199, g))[0]

In [23]:
trans_genome = list(filter(lambda x: x.key == 199, t))[0]

In [24]:
plot_path([sim_genome, trans_genome])

166